In [1]:
import numpy as np
import pandas as pd
import scipy.io as sc
from sklearn import svm
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from imblearn.over_sampling import SMOTENC,ADASYN,SMOTE
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

from sklearn.tree import DecisionTreeClassifier


In [101]:
#1. SVM with Polynomial Kernel

X_train=sc.loadmat('/Users/ushakiranmayee/Documents/DataMining/Assignments/Assignment 4/Data for Assignment 4/Data for Problem 1/X_train.mat')
Y_train=sc.loadmat('/Users/ushakiranmayee/Documents/DataMining/Assignments/Assignment 4/Data for Assignment 4/Data for Problem 1/Y_train.mat')
X_test=sc.loadmat('/Users/ushakiranmayee/Documents/DataMining/Assignments/Assignment 4/Data for Assignment 4/Data for Problem 1/X_test.mat')
Y_test=sc.loadmat('/Users/ushakiranmayee/Documents/DataMining/Assignments/Assignment 4/Data for Assignment 4/Data for Problem 1/Y_test.mat')

xt=X_train['X_train']
yt=Y_train['y_train']
x_test=X_test['X_test']
y_test=Y_test['y_test']


def SVM_with_different_kernels(name,k):
    total_ypred=[]
    for cls in range(yt.shape[1]):
        #for samples in range(xt.shape[0]):
            model=svm.SVC(kernel=k,degree=2)
            model.fit(xt,yt[:,cls])
            y_predict_per_class=model.predict(x_test)
            #print(y_predict_per_class)
            total_ypred.append(y_predict_per_class)
    total_ypred=np.array(total_ypred)
    total_ypred=total_ypred.T
    #print(total_ypred)
    rows_with_zerost = np.sum(np.all((y_test == 0).all(axis=1)))
    rows_with_zerosp = np.sum(np.all((total_ypred == 0).all(axis=1)))
    #print(rows_with_zerost,rows_with_zerosp )
    
    
    #Calculating Accuracy
    
    Accuracy=[]
    for sample in range(y_test.shape[0]):
        Common=0
        Sum=0
        for cls in range(y_test.shape[1]):
            #print(sample)
            if(total_ypred[sample][cls]==1 and y_test[sample][cls]==1):
                Common+=1
                Sum+=1
            elif((total_ypred[sample][cls]==1 and y_test[sample][cls]==0) or (total_ypred[sample][cls]==0 and y_test[sample][cls]==1)):
                Sum+=1
        #print(Common,Sum,y_test[sample],total_ypred[sample])
        if(Sum==0):
            A=0
        else:
            A=Common/Sum
        Accuracy.append(A)
    Final_Accuracy=np.mean(Accuracy)
    print(f"Accuracy with {name} Kernel: ",Final_Accuracy*100,"%")
    
kernel={'Polynomial':'poly','Gaussian':'rbf'}
for name,val in kernel.items():
    SVM_with_different_kernels(name,val)


Accuracy with Polynomial Kernel:  61.852260198456456 %
Accuracy with Gaussian Kernel:  63.43256155825065 %


In [31]:
#2. K-means Clustering
# Algorithm
# 1. Pick random k clusters k=[3,5,7]
# 2. Calculate the dst of each pt to these clusters
# 3. Assign the nearest cluster to each data point
# 4. Repeat the process for either 100 iterations or change in SSE for iterations is < 0.001 

#1. Loading the Dataset
Data=np.loadtxt('/Users/ushakiranmayee/Documents/DataMining/Assignments/Assignment 4/Data for Assignment 4/Data for Problem 2/seeds.txt')
Data_list=Data.tolist()
#2.1 k=[3,5,7]
def K_Means_Clustering(k):
    SSE_10=[0]*10
    
    for init in range(10):
        #Initializing k random initial clusters
        initial_clusters=random.sample(Data_list, k)
        #print(initial_clusters)
        #SSE=0
        prev_SSE=1<<31
        for iter in range(100):
            #print("iter",iter,init)
            #e=[]
            #initializing k lists to store the data in each cluster respectively
            classification=[[] for j in range(k)] 
            for sample in Data:
                #Calculating the euclidean distance between each sample and each cluster
                e=[np.linalg.norm(sample-clu) for clu in initial_clusters]
                #considering the index within e which has minimum distance value
                index=np.argmin(e)
                
                classification[index].append(sample)
    
            #Got the samples distributed within each cluster. Now calculating the initial centroid of each cluster
            mean_clusters=[np.mean(ele,axis=0) for ele in classification]
            #For datapoint in each cluster calculating its Sq_distance from the mean to the datapoint
            
                
            SSE=np.sum([np.sum((classification[cluster] - mean_clusters[cluster])**2) for cluster in range(k)])
            if(prev_SSE-SSE)<0.001:
                break
            prev_SSE=SSE
            initial_clusters=mean_clusters
            
        SSE_10[init]=SSE
    
    SSE_mean=np.mean(SSE_10)
    print(f"SSE_mean with k={k}", SSE_mean)
    
k_values=[3,5,7]
for samples in range(3):
    K_Means_Clustering(k_values[samples])

SSE_mean with k=3 587.9039638275232
SSE_mean with k=5 410.28892085150017
SSE_mean with k=7 298.57799179993054


In [28]:
#1. Model - Random Forest

def RandomForestClassification(method,X_train_encoded,X_test_encoded,y_resampled):
    rf_model=RandomForestClassifier()
    rf_model.fit(X_train_encoded, y_resampled)
    predict_y_rf=rf_model.predict(X_test_encoded)
    f1=f1_score(yt,predict_y_rf)
    print("1. RandomForest")
    print(f"F1-Score After the Balancing of the dataset by {method} is",f1)

#2. Model - DecisionTree

def DecisionTreeClassification(method,X_train_encoded,X_test_encoded,y_resampled):
    dt_model=DecisionTreeClassifier()
    dt_model.fit(X_train_encoded, y_resampled)
    predict_y_dt=dt_model.predict(X_test_encoded)
    f1=f1_score(yt,predict_y_dt)
    print("2. DecisionTree")
    print(f"F1-Score After the Balancing of the dataset by {method} is",f1)

#3. ADABOOST Classifier

def AdaboostClassification(method,X_train_encoded,X_test_encoded,y_resampled):
    
    adaboost_model = AdaBoostClassifier(n_estimators=50, random_state=42)
    adaboost_model.fit(X_train_encoded, y_resampled)
    predict_y_ada=adaboost_model.predict(X_test_encoded)
    f1=f1_score(yt,predict_y_ada)
    print("3. ADABOOST Classifier")
    print(f"F1-Score After the Balancing of the dataset by {method} is",f1,end="\n\n")


In [29]:
#Problem_3
#attributes=[' Status of existing checking account', 'Duration',' Credit history','Purpose','Credit amount','Savings account/bonds','Present employment since','Installment rate','Personal status','guarantors','Present residence since','Property','Age in years','Other installment plans','Housing','Number of existing credits at this bank','Job','Number of people being liable to provide maintenance for','Telephone','foreign worker']
xdata_3=pd.read_csv("/Users/ushakiranmayee/Documents/DataMining/Assignments/Assignment 4/Data for Assignment 4/Data for Problem 3/German Credit Data.txt",delimiter=",",header=None)

y_3=xdata_3.iloc[:,-1]
x_3=xdata_3.iloc[:,:-1]


x,xt,y,yt=train_test_split(x_3,y_3,test_size=0.3,random_state=42,stratify=y_3)
categorical_features=list(x_3.select_dtypes(include=['object']).columns)

In [146]:
#3.1 Using SMOTENC model for handling the class Imbalance Problem with OneHotEncoder

smote_nc=SMOTENC(categorical_features=categorical_features, random_state=42)
x_resampled, y_resampled=smote_nc.fit_resample(x,y)


#Converting the Categorical Data to Numerical Data for training the models
encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

X_train_encoded = encoder.fit_transform(x_resampled)
X_test_encoded=encoder.fit_transform(xt)

print("Method-SMOTENC",end="\n\n")

RandomForestClassification("SMOTENC",X_train_encoded,X_test_encoded,y_resampled)
DecisionTreeClassification("SMOTENC",X_train_encoded,X_test_encoded,y_resampled)
AdaboostClassification("SMOTENC",X_train_encoded,X_test_encoded,y_resampled)

#3.2 Using ADASYN model for handling the class Imbalance Problem with OneHotEncoder

x_en=encoder.fit_transform(x)
xt_en=encoder.fit_transform(xt)
#2. Adaptive Synthetic Sampling

ADA_SYN_MODEL=ADASYN()
x_resampled1,y_resampled1=ADA_SYN_MODEL.fit_resample(x_en,y)

print("\nMethod-ADASYN",end="\n\n")

RandomForestClassification("ADASYN",x_resampled1,xt_en,y_resampled1)
DecisionTreeClassification("ADASYN",x_resampled1,xt_en,y_resampled1)
AdaboostClassification("ADASYN",x_resampled1,xt_en,y_resampled1)


#3.3 Using SMOTE model for handling the class Imbalance Problem with Label Encoder
label_encoder=LabelEncoder()
for col in categorical_features:
    x[col]=label_encoder.fit_transform(x[col])

for col in categorical_features:
    xt[col]=label_encoder.fit_transform(xt[col])

#print(x)
smote=SMOTE()
x_rs3,y_rs3=smote.fit_resample(x,y)

print("\nMethod-SMOTE",end="\n\n")

RandomForestClassification("SMOTE",x_rs3,xt,y_rs3)
DecisionTreeClassification("SMOTE",x_rs3,xt,y_rs3)
AdaboostClassification("SMOTE",x_rs3,xt,y_rs3)



Method-SMOTENC

1. RandomForest
F1-Score After the Balancing of the dataset by SMOTENC is 0.8341013824884792
2. DecisionTree
F1-Score After the Balancing of the dataset by SMOTENC is 0.74
3. ADABOOST Classifier
F1-Score After the Balancing of the dataset by SMOTENC is 0.801955990220049


Method-ADASYN

1. RandomForest
F1-Score After the Balancing of the dataset by ADASYN is 0.8296943231441049
2. DecisionTree
F1-Score After the Balancing of the dataset by ADASYN is 0.7167919799498748
3. ADABOOST Classifier
F1-Score After the Balancing of the dataset by ADASYN is 0.8094117647058824


Method-SMOTE

1. RandomForest
F1-Score After the Balancing of the dataset by SMOTE is 0.8259860788863109
2. DecisionTree
F1-Score After the Balancing of the dataset by SMOTE is 0.7707317073170733
3. ADABOOST Classifier
F1-Score After the Balancing of the dataset by SMOTE is 0.8229665071770335

